# Initialization vs Isometry in Qiskit

According to `201 Tips and tricks.ipynb` there are exist two approaches to prepare the state: one was proposed by Shende, Bullock and Markov in ["Synthesis of Quantum Logic Circuits"](https://arxiv.org/abs/quant-ph/0406176) (2004) and another — by Iten et al. in ["Quantum Circuits for Isometries"](https://arxiv.org/abs/1501.06911) (2020). For more details please refer to correspondent notebook and original papers.

This notebook concerns comparison of these two methods in terms of how close obtained states are to the desired vector. For someone who knows quantum programming inside out, the answer might appear trivial. Nevertheless, let's give it a try.

### 0. Prerequisites

In [1]:
import math
import numpy as np

from qiskit import QuantumCircuit, BasicAer, execute, transpile, IBMQ
from scipy.stats import wasserstein_distance as ws
from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm

IBMQ.load_account()

import warnings
warnings.filterwarnings("ignore")

### 1. Metrics

Let's construct a class that will examine how emperical distribution is close to an input vector. Since, we operate with arrays of numbers of a fixed length, we can consider them both as distributions and as vectors. Methods will be assessed in terms of [mean-absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error), [Wasserstein metric](https://en.wikipedia.org/wiki/Wasserstein_metric) — minimal amount of work to transform one distribution into another, and [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) — measure of how orientation and direction of vectors is close.

In [2]:
class Metrics:
    
    def __init__(self):
        self.metrics_available = ["mae", "wasserstein", "cosine_similarity", "n_gates"]
    
    def compute(self, qc, data, dist):
        metrics = dict(
            mae = np.abs(data - dist).mean(),
            wasserstein = ws(data, dist),
            cosine_similarity = np.dot(data, dist) / (np.linalg.norm(data) * np.linalg.norm(dist)),
            n_gates = qc.count_ops()['cx']
        )
        return metrics

### 2. Baсkends

Experiments will be conducted both in simulators and on a QPU. Thus, it makes sense to create a separate class that "unifies" creation of new backend objects.

In [3]:
class BackendFactory:
    
    def __init__(self):
        pass
    
    def get_new(self):
        pass
    
    def __str__(self):
        pass

In [4]:
class QASMSimulatorFactory(BackendFactory):
    
    def __init__(self):
        super().__init__()
        pass
    
    def get_new(self):
        return BasicAer.get_backend('qasm_simulator')
    
    def __str__(self):
        return 'qasm_simulator'

In [5]:
class IBMQpuFactory(BackendFactory):
    
    def __init__(self, name="ibmq_belem"):
        super().__init__()
        self.provider = IBMQ.get_provider('ibm-q')
        self.name = name
    
    def get_new(self):
        return self.provider.get_backend(self.name)
    
    def __str__(self):
        return f'ibm-q {self.name}'

In [6]:
provider = IBMQ.get_provider('ibm-q')
available_cloud_backends = provider.backends() 
for backend in available_cloud_backends:
    status = backend.status()
    is_operational = status.operational
    jobs_in_queue = status.pending_jobs
    print(f"{backend}\t is online={is_operational}\twith a queue={jobs_in_queue}")

ibmq_qasm_simulator	 is online=True	with a queue=2
ibmq_lima	 is online=True	with a queue=370
ibmq_belem	 is online=True	with a queue=160
ibmq_quito	 is online=True	with a queue=27
simulator_statevector	 is online=True	with a queue=2
simulator_mps	 is online=True	with a queue=3
simulator_extended_stabilizer	 is online=True	with a queue=3
simulator_stabilizer	 is online=True	with a queue=2
ibmq_manila	 is online=True	with a queue=119
ibm_nairobi	 is online=True	with a queue=510
ibm_oslo	 is online=True	with a queue=260


### 3. Design single experiment

Our experiment consists of several stages: 
1. Prepare a random vector of size n
2. Create two quantum circuits with `.initialize()` and `.isometry()` methods
3. Execute quantum circuits
4. Measure the resultant distribution
5. Assess how close is this distribution to the desired one

In [7]:
class Experiment:
    
    def __init__(self, backend_factory, n_repetitions=1, n_qubits=4, shots=65535):
        self.backend_factory = backend_factory
        self.n_repetitions = n_repetitions
        self.n_qubits = n_qubits
        self.vector_size = 2 ** n_qubits
        self.shots = shots
        
        self.data = self.__get_random_vector()
        self.metrics = Metrics()
        
    def run(self):
        results = list()
        data = self.data
        for rep in tqdm(range(self.n_repetitions)):  
            init_qc = self.__prepare_initialize_circuit(data)
            isom_qc = self.__prepare_isometry_circuit(data)

            result = dict(
                initialize = self.__apply(init_qc, data, self.shots),
                isometry = self.__apply(isom_qc, data, self.shots)
            )
            results.append(result)
        return results
    
    def __prepare_initialize_circuit(self, data):
        n = self.n_qubits
        range_n = range(n)
        qc = QuantumCircuit(n, n)
        qc.initialize(data)
        qc.measure(range_n, range_n)
        return qc
    
    def __prepare_isometry_circuit(self, data):
        n = self.n_qubits
        range_n = range(n)
        qc = QuantumCircuit(n, n)
        qc.isometry(data, list(range_n), None)
        qc.measure(range_n, range_n)
        return qc
    
    def __apply(self, qc, data, shots):
        backend = self.backend_factory.get_new()
        qc = transpile(qc, backend)
        counts = execute(qc, backend, shots=shots).result().get_counts()
        dist = self.__get_dist(counts)
        metrics = self.metrics.compute(qc, data, dist)
        return metrics
    
    def __get_dist(self, counts):
        dist = [0. for i in range(self.vector_size)]
        for a, b in sorted(list(counts.items())):
            dist[int(a, 2)] = b
        dist = np.array(dist)
        dist = dist / np.linalg.norm(dist)
        return dist
        
    def __get_random_vector(self):
        a = np.random.rand(self.vector_size)
        a = a / np.linalg.norm(a)
        return a

In [8]:
experiment_sim = Experiment(
    backend_factory=QASMSimulatorFactory(),
    n_repetitions=1,
    n_qubits=5, 
    shots=65535
)

Let's run experiments both in simulator and QPU for sanity checking.

In [9]:
experiment_sim.run()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.76it/s]


[{'initialize': {'mae': 0.04253919412604212,
   'wasserstein': 0.04253919412604211,
   'cosine_similarity': 0.9634270634527289,
   'n_gates': 30},
  'isometry': {'mae': 0.043300179474010624,
   'wasserstein': 0.043300179474010624,
   'cosine_similarity': 0.9628786100813833,
   'n_gates': 26}}]

In [10]:
experiment_ibm = Experiment(
    backend_factory=IBMQpuFactory(name="ibmq_quito"),
    n_repetitions=1,
    n_qubits=5, 
    shots=20000 # max. number of shots for ibm-q
)

In [ ]:
experiment_ibm.run()

  0%|                                                                                                              | 0/1 [00:00<?, ?it/s]

### 4. Analyse multiple experiments

Processes in quantum computations are stochastic. So, in order to draw any conclusions, we should (at least try to) conduct multiple experiments. Frankly speaking, we might want to run experiments until the vector averaged over all iterations converges. The bad thing is that I don't have my own QPU yet :( , so we will wait a lot in a queue. Also, this approach inherits problems with floating-point arithmetic. Therefore, let the number of iterations (=experiments) be fixed.

In [10]:
class Inspector:
    
    def __init__(self):
        self.metrics_available = Metrics().metrics_available
    
    def run(self, results, name):
        metrics_dict_init = {m: list() for m in self.metrics_available}
        metrics_dict_isom = {m: list() for m in self.metrics_available}
        
        for result in results:
            for m, v in result["initialize"].items():
                metrics_dict_init[m].append(v)
               
            for m, v in result["isometry"].items():
                metrics_dict_isom[m].append(v)
        
        print(f"Backend: {name}")
        
        print("  'qc.initialise':")
        for m, v in metrics_dict_init.items():
            v = np.array(v)
            print(f"\t{m}: {v.mean():.8f} ± {v.var():.8f}")
        
        print("  'qc.isometry':")
        for m, v in metrics_dict_isom.items():
            v = np.array(v)
            print(f"\t{m}: {v.mean():.8f} ± {v.var():.8f}")
         
        

In [11]:
inspector = Inspector()

In [12]:
experiment_sim = Experiment(
    backend_factory=QASMSimulatorFactory(),
    n_repetitions=1000,
    n_qubits=5, 
    shots=65535
)

results_sim = experiment_sim.run()
inspector.run(results_sim, "qasm_simulator")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.74it/s]

Backend: qasm_simulator
  'qc.initialise':
	mae: 0.04218623 ± 0.00000042
	wasserstein: 0.04218623 ± 0.00000042
	cosine_similarity: 0.96248341 ± 0.00000189
	n_gates: 30.00000000 ± 0.00000000
  'qc.isometry':
	mae: 0.04186775 ± 0.00000043
	wasserstein: 0.04186775 ± 0.00000043
	cosine_similarity: 0.96275767 ± 0.00000070
	n_gates: 26.00000000 ± 0.00000000


In [ ]:
experiment_ibm = Experiment(
    backend_factory=IBMQpuFactory(name="ibmq_quito"),
    n_repetitions=5,
    n_qubits=5, 
    shots=20000 # max. number of shots for ibm-q
)

results_ibm = experiment_ibm.run()
inspector.run(results_ibm, "ibmq_quito")

### Discussion

The results show that both methods are of compatible accuracy in terms of MAE, Wasserstein metric and cosine similariy. Although, `.isometry()` gives a slightly better performance. The reason is probably in the lower number of gates used by isometry and consequent better fidelity of an obtained state vector. 

Due to the fact that experiments take much time, I did not observe the relationship between number of qubits in use and accuracy of methods. Thus, I would assume that there is a linear dependency and results will be the same for lower number of qubits, albeit this is a topic for further study. 